In [5]:
import onnx
import tvm
from tvm import relay
from tvm.contrib import graph_executor
import numpy as np

In [6]:
onnx_model_path = "mnist_resnet_final.onnx"
onnx_model = onnx.load(onnx_model_path)

In [7]:
input_name = "input"
input_shape = (1, 224, 224, 3)
input_dtype = "float32"

In [8]:
target = "llvm"
shape_dict = {input_name: input_shape}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

In [9]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [10]:
dev = tvm.device(target, 0)  # Use 0 for the first device
module = graph_executor.GraphModule(lib["default"](dev))

In [11]:
input_data = np.random.randn(*input_shape).astype(input_dtype)
module.set_input(input_name, input_data)

In [12]:
module.run()

In [13]:
output_index = 0  # Replace with the correct output index if there are multiple outputs
output = module.get_output(output_index).asnumpy()
print("Output shape:", output.shape)

Output shape: (1, 10)


In [15]:
compiled_model_path = "tvm _model.so"
lib.export_library(compiled_model_path)

In [16]:
print(mod)

def @main(%input: Tensor[(1, 224, 224, 3), float32] /* ty=Tensor[(1, 224, 224, 3), float32] span=model/conv2d/BiasAdd__6.input:0:0 */) -> Tensor[(1, 10), float32] {
  %0 = transpose(%input, axes=[0, 3, 1, 2]) /* ty=Tensor[(1, 3, 224, 224), float32] span=model/conv2d/BiasAdd__6:0:0 */;
  %1 = nn.conv2d(%0, meta[relay.Constant][0] /* ty=Tensor[(64, 3, 7, 7), float32] span=model/conv2d/BiasAdd.model/conv2d/Conv2D/ReadVariableOp:0:0:0 */, strides=[2, 2], padding=[2, 2, 3, 3], channels=64, kernel_size=[7, 7]) /* ty=Tensor[(1, 64, 112, 112), float32] span=model/conv2d/BiasAdd:0:0 */;
  %2 = nn.bias_add(%1, meta[relay.Constant][1] /* ty=Tensor[(64), float32] span=model/conv2d/BiasAdd.model/conv2d/BiasAdd/ReadVariableOp:0:0:0 */) /* ty=Tensor[(1, 64, 112, 112), float32] span=model/conv2d/BiasAdd:0:0 */;
  %3 = nn.relu(%2) /* ty=Tensor[(1, 64, 112, 112), float32] span=model/conv2d/Relu:0:0 */;
  %4 = nn.batch_norm(%3, meta[relay.Constant][2] /* ty=Tensor[(64), float32] span=model/batch_normaliz

In [17]:
with open("compiled_graph.json", "w") as f:
    f.write(lib.get_graph_json())